# Similitud entre productos

Descripción
Un desafío constante en MELI es el de poder agrupar productos similares utilizando algunos
atributos de estos como pueden ser el título, la descripción o su imagen.
Para este desafío tenemos un dataset “items_titles.csv” que tiene títulos de 30 mil productos
de 3 categorías diferentes de Mercado Libre Brasil.
El objetivo del desafío es poder generar una Jupyter notebook que determine cuán similares
son dos títulos del dataset “item_titles_test.csv” generando como output un listado de la
forma de tabla donde ordenando por score de similitud podamos encontrar los pares de productos más
similares en nuestro dataset de test.

In [1]:
# bibliotecas
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import gower

In [2]:
# Importação de tabelas
titles = pd.read_csv('items_titles.csv')
items = pd.read_csv('items_titles_test.csv')

In [3]:
titles

,ITE_ITEM_TITLE
0,Tênis Ascension Posh Masculino - Preto E Verme...
1,Tenis Para Caminhada Super Levinho Spider Corr...
2,Tênis Feminino Le Parc Hocks Black/ice Origina...
3,Tênis Olympikus Esportivo Academia Nova Tendên...
4,Inteligente Led Bicicleta Tauda Luz Usb Bicicl...
...,...
29995,Tênis Vans Old Skool I Love My Vans - Usado - ...
29996,Tênis Feminino Preto Moleca 5296155
29997,Tenis Botinha Com Pelo Via Marte Original Lanç...
29998,Tênis Slip On Feminino Masculino Original Sapa...


In [4]:
items

,ITE_ITEM_TITLE
0,Tênis Olympikus Esporte Valente - Masculino Kids
1,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...
2,Tênis Usthemp Slip-on Temático - Labrador 2
3,Tênis Casual Feminino Moleca Tecido Tie Dye
4,Tênis Star Baby Sapatinho Conforto + Brinde
...,...
9995,Chuteira Futsal Oxn Velox 3 Infantil
9996,Sapatenis Casual Masculino Estiloso 24horas Co...
9997,Tênis Feminino Infantil Molekinha Tie Dye
9998,Tênis Feminino Leve Barato Ganhe 1 Colchonete ...


### Função de limpeza

In [5]:
# Remover caracteres especiais, transformar o texto em letra minúscula e remover os acentos das palavras
def clean_function(titles):
    titles['ITE_ITEM_TITLE'] = titles[['ITE_ITEM_TITLE']]\
    .replace(regex=r'[!/,.-]',value='')\
    .apply(lambda x: x.astype(str).str.lower())\
    .apply(lambda x:x.astype(str).str.normalize('NFKD').str.encode('ASCII', errors= 'ignore').str.decode('utf-8'))
    return titles

In [6]:
new_titles = clean_function(titles)
new_titles

,ITE_ITEM_TITLE
0,tenis ascension posh masculino preto e vermelho
1,tenis para caminhada super levinho spider corr...
2,tenis feminino le parc hocks blackice original...
3,tenis olympikus esportivo academia nova tenden...
4,inteligente led bicicleta tauda luz usb bicicl...
...,...
29995,tenis vans old skool i love my vans usado fe...
29996,tenis feminino preto moleca 5296155
29997,tenis botinha com pelo via marte original lanc...
29998,tenis slip on feminino masculino original sapa...


In [7]:
new_items = clean_function(items)
new_items

,ITE_ITEM_TITLE
0,tenis olympikus esporte valente masculino kids
1,bicicleta barra forte samy c 6 marchas cubo c ...
2,tenis usthemp slipon tematico labrador 2
3,tenis casual feminino moleca tecido tie dye
4,tenis star baby sapatinho conforto + brinde
...,...
9995,chuteira futsal oxn velox 3 infantil
9996,sapatenis casual masculino estiloso 24horas co...
9997,tenis feminino infantil molekinha tie dye
9998,tenis feminino leve barato ganhe 1 colchonete ...


### Remoção de Stop Words

In [8]:
# Remover palavras que não auxiliam no entendimento da frase
def stop_words_function(new_titles):
    stop_words = stopwords.words('portuguese')
    new_titles['ITE_ITEM_TITLE'] = new_titles['ITE_ITEM_TITLE'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    return new_titles

In [9]:
new_titles = stop_words_function(new_titles)
new_titles

,ITE_ITEM_TITLE
0,tenis ascension posh masculino preto vermelho
1,tenis caminhada super levinho spider corrida
2,tenis feminino le parc hocks blackice original...
3,tenis olympikus esportivo academia nova tenden...
4,inteligente led bicicleta tauda luz usb bicicl...
...,...
29995,tenis vans old skool i love my vans usado femi...
29996,tenis feminino preto moleca 5296155
29997,tenis botinha via marte original lancamento
29998,tenis slip on feminino masculino original sapa...


In [10]:
def stop_words(new_items):
    stop_words = stopwords.words('portuguese')
    new_items['ITE_ITEM_TITLE'] = new_items['ITE_ITEM_TITLE'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    return new_items

In [11]:
new_items = stop_words(new_items)
new_items

,ITE_ITEM_TITLE
0,tenis olympikus esporte valente masculino kids
1,bicicleta barra forte samy c 6 marchas cubo c ...
2,tenis usthemp slipon tematico labrador 2
3,tenis casual feminino moleca tecido tie dye
4,tenis star baby sapatinho conforto + brinde
...,...
9995,chuteira futsal oxn velox 3 infantil
9996,sapatenis casual masculino estiloso 24horas co...
9997,tenis feminino infantil molekinha tie dye
9998,tenis feminino leve barato ganhe 1 colchonete ...


### Remoção de números

In [12]:
new_titles = new_titles.replace('\d+', '', regex=True)
new_titles

,ITE_ITEM_TITLE
0,tenis ascension posh masculino preto vermelho
1,tenis caminhada super levinho spider corrida
2,tenis feminino le parc hocks blackice original...
3,tenis olympikus esportivo academia nova tenden...
4,inteligente led bicicleta tauda luz usb bicicl...
...,...
29995,tenis vans old skool i love my vans usado femi...
29996,tenis feminino preto moleca
29997,tenis botinha via marte original lancamento
29998,tenis slip on feminino masculino original sapa...


In [13]:
new_items = new_items.replace('\d+', '', regex=True)
new_items

,ITE_ITEM_TITLE
0,tenis olympikus esporte valente masculino kids
1,bicicleta barra forte samy c marchas cubo c r...
2,tenis usthemp slipon tematico labrador
3,tenis casual feminino moleca tecido tie dye
4,tenis star baby sapatinho conforto + brinde
...,...
9995,chuteira futsal oxn velox infantil
9996,sapatenis casual masculino estiloso horas conf...
9997,tenis feminino infantil molekinha tie dye
9998,tenis feminino leve barato ganhe colchonete p...


### Correspondência Difusa - Algoritmo de Levenshtein

In [14]:
# criando as listas para armazenar os valores
mat1 = []
mat2 = []
mat3 = []
p = []

# converter os df em listas
list1 = new_titles['ITE_ITEM_TITLE'].tolist()
list2 = new_items['ITE_ITEM_TITLE'].tolist()

In [ ]:
# threshold a no mínimo 85% de semelhança
threshold = 85

# iremos iterar os itens da lista1 para extrair sua correspondência mais próxima da lista2
for i in list1:
    mat1.append(process.extract(i, list2, limit=1))
new_titles['ITE_ITEM_TITLE'] = mat1
new_titles['score'] = mat1
  
new_titles

In [ ]:
# iterar novamente através da coluna de correspondências no loop externo
# no loop interno iteramos através de cada conjunto de correspondências
for j in new_titles['ITE_ITEM_TITLE']:
    for k in j:
        
        if k[1] >= threshold:
            p.append(k[0])
              
    mat2.append(",".join(p))
    p = []
      
# gravando os resultados no dataframe
new_titles['ITE_ITEM_TITLE'] = mat2
  
new_titles

In [ ]:
# score colunm
for j in new_titles['score']:
    for k in j:
        
        if k[1] >= threshold:
            p.append(k[0])
              
    mat3.append(k)
    p = []
    
new_titles['score'] = mat3

new_titles